# Montu Python 
## Astronomical ephemerides for the Ancient World
## Test: Template

In [50]:
import montu
from montu import D2H,PRINTDF,TABLEDF
import pandas as pd
import numpy as np

# Load legacy code to compare
import montu.__cycle_3 as montu2
montu2.Util.load_kernels()

# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Testing calendario

Dates around 1 CE:

In [416]:
#montu.Time('0-1-1',calendar='proleptic')
montu.Time('240-01-01 12:00:00',calendar='mixed')

Time('0240-01-01 12:00:00.0000'/'240-01-01 12:00:00'/'hrw 3022-III-Peret-11'/JED 1808718.0/JTD 1808718.0955845)

In [417]:
montu.Time('139-07-20 00:00:00',calendar='mixed')

Time('0139-07-19 00:00:00.0000'/'139-07-20 00:00:00'/'hrw 2922-I-Akhet-1'/JED 1772027.5/JTD 1772027.6066806)

In [423]:
montu.Time('h 2922-I-Akhet-1 18:00:00',calendar='caniucular')

Time('0139-07-19 18:00:00.0000'/'139-07-20 18:00:00'/'hrw 2922-I-Akhet-1'/JED 1772028.25/JTD 1772028.3566806)

In [420]:
%timeit montu.Time('hrw 2922-I-Akhet-1 12:00:00',calendar='caniucular')

201 µs ± 9.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Lull page 95:

In [81]:
mtime1 = montu.Time('240-06-01',calendar='mixed')
mtime2 = montu.Time('44 bce 01-01',calendar='mixed')
mtime1.sub(mtime2)/365.25

283.4140999315537

In [82]:
(mtime1.readable.obj_datetime64 - mtime2.readable.obj_datetime64)/(1e6*montu.CALYEAR)

numpy.timedelta64(283,'us')

In [35]:
montu.Time('0-06-01 12:00:00',calendar='mixed')

Time('-0000-05-30 12:00:00.0000'/'0-06-01 12:00:00'/JED 1721210.0/JTD 1721210.1224421)

Year of *apokatastasis*: 139 c.e.

In [120]:
mtime_apk1 = montu.Time('139-06-01 12:00:00',calendar='mixed')
mtime_apk1

Time('0139-05-31 12:00:00.0000'/'139-06-01 12:00:00'/JED 1771979.0/JTD 1771979.1066898)

In [122]:
mtime_apk2 = mtime_apk1.sub(1461*montu.CALYEAR).get_readable()
mtime_apk2

Time('-1321-05-20 12:00:00.0000'/'-1321-06-01 12:00:00'/JED 1238714.0/JTD 1238714.3651713)

In [125]:
1461*montu.CALYEAR, 1460*montu.JULYEAR

(46074096000, 46074096000.0)

In [118]:
mtime_apk2.diff(mtime_apk1)/365.25

-1460.0

In [126]:
365*4

1460

In [114]:
mtime_apk3 = mtime_apk2.sub(1460*montu.JULYEAR).get_readable()
mtime_apk3

Time('-2781-05-09 12:00:00.0000'/'-2781-06-01 12:00:00'/JED 705449.0/JTD 705449.7838125)

In [115]:
mtime_apk4 = mtime_apk3.sub(1460*montu.JULYEAR).get_readable()
mtime_apk4


Time('-4241-04-28 12:00:00.0000'/'-4241-06-01 12:00:00'/JED 172184.0/JTD 172185.3603507)

In [349]:
HORUS_MONTH = dict(I=1,II=2,III=3,IV=4)
HORUS_SEASON = dict(AKHET=1,PERET=2,SHEMU=3,
                    A=1,P=2,S=3,
                    AKH=1,PER=2,SHE=3)
MONTH_HORUS = {str(v): k for k, v in HORUS_MONTH.items()}
SEASON_HORUS = {'0':'Mesut','1':'Akhet','2':'Peret','3':'Shemu'}

def _horus_days(horus=0,month='I',season='Akhet',day=1):
    """Obtain the number of days since I-Akhet-1 of the
    year of the first ancient egyptian apokotástasis (2782 b.c.e.).

    Parameters:
        horus: int, default = 0:
            Horus year, i.e. civil year starting at 2782 bce.
        
        month: string, default = 'I':
            Month in the season. Values are I, II, III, IV

        season: string, default = 'Akhet':
            Season of the year. 
            Values are AKHET (A,AKH), PERET (P,PER), SHEMU (S,SHE).

        day: int, default = 1:
            Day. It must be between 1 and 30 (including).

    Return:
        Number of days since 2782 bce I-Akhet-1.
    """
    D = (int(day)-1) + (HORUS_SEASON[season]-1)*120 + (HORUS_MONTH[month]-1)*30 + int(horus)*365
    return D

def _horus_date_to_days(date):
    """Obtain the number of days since I-Akhet-1 of the
    year of the first ancient egyptian apokotástasis (2782 b.c.e.).

    Parameters:
        date: string:
            Format: CCYY-MM-SS-DD

            where: 
                CCYY: Horus year, 0 = 2782 bce.
                MM: Number of month (I, II, III, IV)
                SS: Name (letter of abreviation) of season:
                    Akhet (akh,a), Peret (per,p), Shemu (she,s)
                DD: Number of day

    Return:
        hd: int:
            Horus days. Number of days since 2782 bce I-Akhet-1.
    """
    comps = date.split('-')
    
    # Adjust ranges
    comps[0] = int(comps[0].strip('hHrw ')) # Horus year
    comps[1] = comps[1].upper() # Month (I, II, III, IV)
    comps[2] = comps[2].upper() # Season (akhet,peret,shemu)
    comps[3] = int(comps[3]) # Day (1..30)

    if comps[1] not in HORUS_MONTH.keys():
        raise ValueError(f"Month '{comps[1]}' not recognized in '{date}', it must be among {tuple(HORUS_MONTH.keys())}")
    if comps[2] not in HORUS_SEASON.keys():
        raise ValueError(f"Season '{comps[2]}' not recognized in '{date}', it must be among {tuple(HORUS_SEASON.keys())}")
    if (int(comps[3])>30) or (int(comps[3])<1):
        raise ValueError(f"Day '{comps[3]}' out of range. It must be between [1,30]")

    return _horus_days(*comps)

def _days_to_horus_date(hdays):
    """Convert from Horus days to civil date
    """

In [353]:
(_horus_days(0,'I','A',1),
_horus_days(0,'II','A',1),
_horus_days(0,'III','P',1),
_horus_date_to_days('0-III-P-1'),
_horus_date_to_days('hrw 1000-III-P-1'),
)

(0, 30, 180, 180, 365180)

In [348]:
'hrw 1000'.strip('hHrw ')

'1000'

In [387]:
mtime_apk1 = montu.Time('139-07-20 00:00:00',calendar='mixed')
mtime_apk1

Time('0139-07-19 00:00:00.0000'/'139-07-20 00:00:00'/JED 1772027.5/JTD 1772027.6066806)

In [388]:
mtime_apk2 = mtime_apk1.sub(2*1461*montu.CALYEAR).get_readable()
mtime_apk2

Time('-2781-06-27 00:00:00.0000'/'-2781-07-20 00:00:00'/JED 705497.5/JTD 705498.2838125)

In [389]:
mtime_apk2.jtd

705498.2838125

In [390]:
print(mtime_apk2.get_readable())

Montu Time Object:
-------------------------- 
Readable:
    Date in proleptic UTC: -2781-06-27 00:00:00.0000
    Date in mixed UTC: -2781-07-20 00:00:00
    Date in SPICE format: 2782 B.C. 06-27 00:00:00.00
    Date in caniucular format: hrw 0-I-Akhet-1
    Components: [-1, 2781, 6, 27, 0, 0, 0, 0]
Objects:
    Date in datetime64 format: -2781-06-27T00:00:00.000000
    Date in PyPlanet Epoch: 705497.5
    Date in PyEphem Epoch: -2782/7/20 00:00:00
General:
    Is bce: True
    Is Julian: True
Uniform scales:
    Terrestrial time:
        tt: -150858436278.6
        jtd: 705498.2838125
        htd: 0.7838124999543652
    UTC time:
        et: -150858504000.0
        jed: 705497.5
        hed: 0.0
    Delta-t = TT - UTC = 67721.4



In [386]:
montu.JED_APOKATASTASIS-montu.JTD_APOKATASTASIS

AttributeError: module 'montu' has no attribute 'JED_APOKATASTASIS'

In [282]:
JED_APOKATASTASIS = 705497.5

In [283]:
mtime = montu.Time('-2781-07-20 00:00:00',calendar='mixed')
mtime

Time('-2781-06-27 00:00:00.0000'/'-2781-07-20 00:00:00'/JED 705497.5/JTD 705498.2838125)

In [284]:
hd = mtime.jed - JED_APOKATASTASIS
hd

0.0

In [285]:
hy = int(hd/365)
dy = hd%365
s = int(dy/120) + 1
ds = dy - 120*(s-1)
m = int(ds/30) + 1
d = int(ds - 30*(m-1)) + 1

hy,dy,s,ds,m,d, str(hy) + "-" + MONTH_HORUS[str(s)] + "-" + SEASON_HORUS[str(s)] + "-" + str(d)

(0, 0.0, 1, 0.0, 1, 1, '0-I-Akhet-1')

In [323]:
JED_APOKATASTASIS = 705497.5

def _horus_day_to_caniucular(hd):
    """Convert from Horus day to caniucular date

    Parameters:
        hd: int:
            Horus days, ie. number of days since
            2782 b.c.e. 07-20, which is the apokatastasis date
            according to the reference date of Censorino
            (see Lull, pag. 96)

    Return:
        cdate: string:
            Caniucular date in the format CCYY-MM-SS-DD where: 
                CCYY: Horus year, 0 = 2782 bce.
                MM: Number of month (I, II, III, IV)
                SS: Name (letter of abreviation) of season:
                    Akhet (akh,a), Peret (per,p), Shemu (she,s)
                DD: Number of day
    """
    hy = int(hd/365) 
    dy = hd%365
    if dy<360:
        s = int(dy/120) + 1
        ds = dy - 120*(s-1)
        m = int(ds/30) + 1
        d = int(ds - 30*(m-1)) + 1
        caniucular = str(hy) + "h-" + MONTH_HORUS[str(m)] + "-" + SEASON_HORUS[str(s)] + "-" + str(d)
    else:
        # Epagomenos
        d = int(dy - 360 + 1)
        caniucular = str(hy) + "h-" + "I" + "-" + SEASON_HORUS['0'] + "-" + str(d)
    return caniucular

def _mixed_to_caniucular(datemix):
    """Convert a mixed date to caniucular
    """
    # Create montu.Time object
    mtime = montu.Time(datemix,calendar='mixed')

    # Compute yhe horus day
    hd = mtime.jed - JED_APOKATASTASIS

    # Compute the caniucular date
    cdate = _horus_day_to_caniucular(hd)
    return cdate


In [324]:
mtime = montu.Time('-2781-07-20 00:00:00',calendar='mixed')
hd = mtime.jed - JED_APOKATASTASIS
_horus_day_to_caniucular(hd)

'0h-I-Akhet-1'

In [335]:
(_mixed_to_caniucular('bce 2782-07-20'),#Apokatastasis I
_mixed_to_caniucular('bce 1322-07-20'),#Apokatastasis II
_mixed_to_caniucular('139-07-20'),#Apokatastasis III
_mixed_to_caniucular('bce 688-07-20'),#Lull, 105
_mixed_to_caniucular('bce 323-06-10'),#Lull, 96
_mixed_to_caniucular('bce 324-11-12'),#Lull, 96
_mixed_to_caniucular('bce 237-08-23'),#Lull, 96
_mixed_to_caniucular('bce 212-08-17'),#Lull, 96
_mixed_to_caniucular('bce 238-03-07'),#Lull, 97
_mixed_to_caniucular('384-07-23'),#Lull, 98
)

('0h-I-Akhet-1',
 '1461h-I-Akhet-1',
 '2922h-I-Akhet-1',
 '2095h-II-Peret-10',
 '2460h-IV-Peret-1',
 '2460h-I-Akhet-1',
 '2546h-III-Shemu-7',
 '2571h-III-Shemu-7',
 '2545h-I-Peret-17',
 '3167h-III-Akhet-6')

In [358]:
montu.Time(montu.Time._horus_date_to_days('hrw 1461-I-Akhet-1') + JED_APOKATASTASIS,format='jd').get_readable()

Time('-1321-07-08 00:00:00.0000'/'-1321-07-20 00:00:00'/JED 1238762.5/JTD 1238762.8651713)

In [373]:
mtime = montu.Time('-2781-07-20 12:00:00',calendar='mixed')
print(mtime)

Montu Time Object:
-------------------------- 
Readable:
    Date in proleptic UTC: -2781-06-27 12:00:00.0000
    Date in mixed UTC: -2781-07-20 12:00:00
    Date in SPICE format: 2782 B.C. 06-27 12:00:00.00
    Date in caniucular format: hrw 0-I-Akhet-1
    Components: [-1, 2781, 6, 27, 12, 0, 0, 0]
Objects:
    Date in datetime64 format: -2781-06-27T12:00:00.000000
    Date in PyPlanet Epoch: 705498.0
    Date in PyEphem Epoch: -2782/7/20 12:00:00
General:
    Is bce: True
    Is Julian: True
Uniform scales:
    Terrestrial time:
        tt: -150858393078.6
        jtd: 705498.7838125
    UTC time:
        et: -150858460800.0
        jed: 705498.0
        hd: 0.5
    Delta-t = TT - UTC = 67721.4



In [371]:
mtime.readable.datecan

'hrw 0-I-Akhet-1'

In [369]:
%timeit mtime = montu.Time('-2781-07-20',calendar='mixed')

644 µs ± 121 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [360]:
montu.Time('hrw 0-I-Akhet-1',calendar='caniucular')

ValueError: Error parsing datetime string "hrw 0-I-Akhet-1" at position 0

In [337]:
montu.Time._mixed_to_caniucular('bce 1322-07-20') #Apokatastasis II

'1461h-I-Akhet-1'

In [288]:
mtime2 = mtime.add(1*montu.DAY)
hd = mtime2.jed - JED_APOKATASTASIS
hd, mtime2.jed, _horus_day_to_caniucular(hd)

(1.0, 705498.5, '0-I-Akhet-2')

In [295]:
mtime = montu.Time('-2720-07-20 00:00:00',calendar='mixed')
dates = []
SOTHIC_TO_JULIAN = 365.25/365
for d in np.arange(0,366):
    mtime2 = mtime.add(d*montu.DAY*SOTHIC_TO_JULIAN)
    hd = mtime2.jed - JED_APOKATASTASIS
    cdate = _horus_day_to_caniucular(hd)
    dates += [{
        'Day':d+1,
        'Gregorian':mtime2.get_readable().readable.datemix,
        'Caniucular':cdate,
    }]
TABLEDF(dates)

|   Day | Gregorian            | Caniucular      |
|-------|----------------------|-----------------|
|     1 | -2720-07-20 00:00:00 | 61-I-Akhet-17   |
|     2 | -2720-07-21 00:00:00 | 61-I-Akhet-18   |
|     3 | -2720-07-22 00:01:01 | 61-I-Akhet-19   |
|     4 | -2720-07-23 00:02:02 | 61-I-Akhet-20   |
|     5 | -2720-07-24 00:03:03 | 61-I-Akhet-21   |
|     6 | -2720-07-25 00:04:04 | 61-I-Akhet-22   |
|     7 | -2720-07-26 00:05:05 | 61-I-Akhet-23   |
|     8 | -2720-07-27 00:06:06 | 61-I-Akhet-24   |
|     9 | -2720-07-28 00:07:07 | 61-I-Akhet-25   |
|    10 | -2720-07-29 00:08:08 | 61-I-Akhet-26   |
|    11 | -2720-07-30 00:09:09 | 61-I-Akhet-27   |
|    12 | -2720-07-31 00:10:10 | 61-I-Akhet-28   |
|    13 | -2720-08-01 00:11:11 | 61-I-Akhet-29   |
|    14 | -2720-08-02 00:12:12 | 61-I-Akhet-30   |
|    15 | -2720-08-03 00:13:13 | 61-II-Akhet-1   |
|    16 | -2720-08-04 00:14:14 | 61-II-Akhet-2   |
|    17 | -2720-08-05 00:15:15 | 61-II-Akhet-3   |
|    18 | -2720-08-06 00:16:16 

### Testing lunar phases

Date of festival in day 6 in Egypt (Lull, page 92):

In [441]:
moon = montu.Planet('Moon')
Tebas = montu.Observer(lat=24,lon=33)

mtime = montu.Time('237 bce 08-23 00:00:00',calendar='mixed')
moon.conditions_in_sky(at=mtime,observer=Tebas)
moon, moon.condition.elongation

(Object Moon positions:
 |           tt |         jed | Name   |   RAJ2000 |   DecJ2000 |   RAEpoch |   DecEpoch |   RAGeo |   DecGeo |       el |      az |
 |--------------|-------------|--------|-----------|------------|-----------|------------|---------|----------|----------|---------|
 | -70541396369 | 1.63509e+06 | Moon   |   16.2303 |    -17.139 |   14.1509 |   -9.11631 | 14.1833 | -8.84156 | -56.1375 | 290.625 |'
 Object Moon conditions:
 |           tt |         jed | Name   |      ha |   Vmag |   rise_time |   rise_az |    set_time |   set_az |   transit_time |   transit_el |   elongation |   earth_distance |   sun_distance | is_circumpolar   | is_neverup   |   angsize |   phase |    hlat |    hlon |   hlong |
 |--------------|-------------|--------|---------|--------|-------------|-----------|-------------|----------|----------------|--------------|--------------|------------------|----------------|------------------|--------------|-----------|---------|---------|---------|--

After 25 civil years:

In [442]:
mtime.add(25*montu.CALYEAR).get_readable()

Time('-0211-08-13 00:00:00.0000'/'-211-08-17 00:00:00'/JED 1644218.5/JTD 1644218.6495243)

Which is the date of the celebration in the time of Ptolomeus IV Filopator.

At the sixth day of the lunar cycle the elongation must be:

In [444]:
360/29.5*6

73.22033898305085

In [445]:
mtime = montu.Time('212 bce 08-17 00:00:00',calendar='mixed')
moon.conditions_in_sky(at=mtime,observer=Tebas)
moon.condition.elongation

73.59571075439453

## Computation of lunar phases

First quarters of 2023:

In [475]:
mtime = montu.Time('2023-01-01 00:00:00')
montu.Moon.next_moon_quarters(since=mtime,output='datepro',numquarters=8)

{'full1': ['2023-01-06 23:09:10.8000', 7.907907465007156, 5.96470835711807],
 'last1': ['2023-01-15 02:11:35.3000', 8.126672363374382, 14.091380720492452],
 'new1': ['2023-01-21 20:54:23.1000', 6.7797199985943735, 20.871100719086826],
 'first1': ['2023-01-28 15:20:01.7000', 6.767808315809816, 27.638909034896642],
 'full2': ['2023-02-05 18:29:51.7000', 8.131828685291111, 35.77073772018775],
 'last2': ['2023-02-13 16:01:56.3000', 7.897275378461927, 43.66801309864968],
 'new2': ['2023-02-20 07:06:56.9000', 6.628478765487671, 50.29649186413735],
 'first2': ['2023-02-27 08:06:50.3000', 7.0415904857218266, 57.33808234985918]}

In [476]:
%timeit montu.Moon.next_moon_quarters(since=mtime,output='jed',numquarters=8)

2.17 ms ± 850 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [478]:
mtime = montu.Time('2023-01-01 00:00:00')
pd.DataFrame(montu.Moon.next_moon_quarters(since=mtime,output='datepro',numquarters=8,format='columns'))

,Quarter,Datetime,delta-t,delta-0
0,full,2023-01-06 23:09:10.8000,7.90790746500715613,5.96470835711807013
1,last,2023-01-15 02:11:35.3000,8.12667236337438226,14.09138072049245238
2,new,2023-01-21 20:54:23.1000,6.77971999859437346,20.87110071908682585
3,first,2023-01-28 15:20:01.7000,6.76780831580981612,27.63890903489664197
4,full,2023-02-05 18:29:51.7000,8.13182868529111147,35.77073772018775344
5,last,2023-02-13 16:01:56.3000,7.89727537846192718,43.66801309864968061
6,new,2023-02-20 07:06:56.9000,6.62847876548767090,50.29649186413735151
7,first,2023-02-27 08:06:50.3000,7.04159048572182655,57.33808234985917807


Other routines for the moon:

In [18]:
montu.Util.get_methods(montu.pymeeus_Moon)

['apparent_ecliptical_pos',
 'apparent_equatorial_pos',
 'geocentric_ecliptical_pos',
 'illuminated_fraction_disk',
 'longitude_mean_ascending_node',
 'longitude_mean_perigee',
 'longitude_true_ascending_node',
 'moon_librations',
 'moon_maximum_declination',
 'moon_passage_nodes',
 'moon_perigee_apogee',
 'moon_phase',
 'moon_position_angle_axis',
 'position_bright_limb']

Returning back to the Ptolomei:

In [474]:
moon = montu.Planet('Moon')
Tebas = montu.Observer(lat=24,lon=33)
mtime = montu.Time('237 bce 08-23 00:00:00',calendar='mixed')
quarter_dates = montu.Moon.next_moon_quarters(since=mtime,output='datemix',numquarters=4)
quarter_dates

{'first': ['-236-08-24 19:33:33', 6.695649390108883, 1.8146231442224234],
 'full': ['-236-09-01 05:08:08', 7.399298035539687, 9.21392117976211],
 'last': ['-236-09-09 09:29:29', 8.18161708698608, 17.39553826674819],
 'new': ['-236-09-16 12:56:56', 7.143462132662535, 24.539000399410725]}

In [454]:
montu.Moon.next_moon_quarters(since=mtime.add(25*montu.CALYEAR),output='datemix',numquarters=4)

{'first': ['-211-08-18 11:46:46', 6.7300694161094725, 1.4906561214011163],
 'full': ['-211-08-26 13:08:08', 8.057218122296035, 9.547874243697152],
 'last': ['-211-09-03 11:30:30', 7.931733892066404, 17.479608135763556],
 'new': ['-211-09-10 02:35:35', 6.628545432118699, 24.108153567882255]}

## Developer storage

In [ ]:
def next_moon_quarters(since=None,
                  quarters=['new','first','full','last'],
                  output='jed'):
    
    pyepoch = montu.pymeeus_Epoch(mtime.jed)

    quarter_dates = dict()

    for quarter in quarters:
        quarter_pyepoch = montu.pymeeus_Moon.moon_phase(pyepoch,target=quarter)

        # If before move search 29 days to the future
        if float(quarter_pyepoch) < mtime.jed:
            quarter_pyepoch=montu.pymeeus_Moon.moon_phase(quarter_pyepoch+29,target=quarter)
        if float(quarter_pyepoch) > (mtime.jed + 30):
            quarter_pyepoch=montu.pymeeus_Moon.moon_phase(quarter_pyepoch-58,target=quarter)

        # Extract date
        if output == 'jed':
            return_value = float(quarter_pyepoch)
        elif output == 'date':
            return_value = quarter_pyepoch.get_full_date()
        elif output == 'mtime':
            return_value = montu.Time(float(quarter_pyepoch),format='jd').get_readable()
        elif output == 'datepro':
            return_value = montu.Time(float(quarter_pyepoch),format='jd').get_readable().readable.datepro
        elif output == 'datemix':
            return_value = montu.Time(float(quarter_pyepoch),format='jd').get_readable().readable.datemix
        else:
            raise ValueError(f"Output format '{output}' not recognized (valid are 'jed', 'mtime')")
        quarter_dates[quarter] = return_value

    if output in ['jed','date']:
        quarter_dates = sorted(quarter_dates.items(),key=lambda x:x[1])

    return quarter_dates

In [ ]:
def next_moon_quarters2(since=None,
                  quarters=['new','first','full','last'],
                  output='jed'):
    
    quarter_dates = dict()
    pyepoch = montu.pymeeus_Epoch(mtime.jed)

    nquarters = 0
    
    # Prepare search
    new_pyepoch = montu.pymeeus_Moon.moon_phase(pyepoch-15,target='new')
    next_pyepoch = new_pyepoch
    prev_pyepoch = next_pyepoch

    print("Closest new':",montu.Time(float(next_pyepoch),format='jd').get_readable().readable.datepro)
    n = 1
    while nquarters<4:
        # Which is the next wuarter
        quarter = quarters[n%4]
        if quarter == 'new':
            new_pyepoch = next_pyepoch
        
        print(f"Reference:",montu.Time(float(new_pyepoch),format='jd').get_readable().readable.datepro)
        
        # Search for next quarters
        next_pyepoch = montu.pymeeus_Moon.moon_phase(new_pyepoch,target=quarter)
        print(f"\tNext quarter '{quarter}':",montu.Time(float(next_pyepoch),format='jd').get_readable().readable.datepro)
        delta_quarter = float(next_pyepoch - prev_pyepoch)
        print(f"\tTime to previous:",delta_quarter)

        # Check if quarter is posterior
        if float(next_pyepoch) > mtime.jed:
            delta_day = float(next_pyepoch - mtime.jed)
            print("\t\tStored")
            # Extract date
            if output == 'jed':
                return_value = float(next_pyepoch)
            elif output == 'date':
                return_value = next_pyepoch.get_full_date()
            elif output == 'mtime':
                return_value = montu.Time(float(next_pyepoch),format='jd').get_readable()
            elif output == 'datepro':
                return_value = montu.Time(float(next_pyepoch),format='jd').get_readable().readable.datepro
            elif output == 'datemix':
                return_value = montu.Time(float(next_pyepoch),format='jd').get_readable().readable.datemix
            else:
                raise ValueError(f"Output format '{output}' not recognized (valid are 'jed', 'mtime')")
            quarter_dates[quarter] = [return_value,delta_quarter,delta_day]
            nquarters += 1
        
        # Sum 
        prev_pyepoch = next_pyepoch
        n += 1
        
    return quarter_dates

first full moon of a year

In [ ]:
mtime = montu.Time('2022-12-01 00:00:00')
pyepoch = montu.pymeeus_Epoch(mtime.jed)
for quarter in 'new','first','full','last':
    quarter_pyepoch = montu.pymeeus_Moon.moon_phase(pyepoch,target=quarter)
    mtime_quarter = montu.Time(float(quarter_pyepoch),format='jd').get_readable()
    print(f"Quarter '{quarter}': {mtime_quarter.readable.datepro}")

Quarter 'new': 2022-11-23 22:58:19.1000
Quarter 'first': 2022-11-30 14:37:39.1000
Quarter 'full': 2022-12-08 04:09:22.1000
Quarter 'last': 2022-12-16 08:57:19.8000


This is the closest but I want the next

In [ ]:
mtime = montu.Time('2022-12-01 00:00:00')
pyepoch = montu.pymeeus_Epoch(mtime.jed)
quarter_pyepoch = montu.pymeeus_Moon.moon_phase(pyepoch,target='new')
print(f"Quarter '{quarter}': {mtime_quarter.readable.datepro}")

Quarter 'last': 2022-12-16 08:57:19.8000


In [ ]:
mtime = montu.Time('2023-01-01 00:00:00')
pyepoch = montu.pymeeus_Epoch(mtime.jed)

quarter = 'new'
quarter = 'full'
quarter_pyepoch = montu.pymeeus_Moon.moon_phase(pyepoch,target=quarter)
if float(quarter_pyepoch) < mtime.jed:
    quarter_pyepoch=montu.pymeeus_Moon.moon_phase(quarter_pyepoch+29,target=quarter)
if float(quarter_pyepoch) > (mtime.jed + 29.5):
    quarter_pyepoch=montu.pymeeus_Moon.moon_phase(quarter_pyepoch-37,target=quarter)

mtime_quarter = montu.Time(float(quarter_pyepoch),format='jd').get_readable()
print(f"Quarter '{quarter}': {mtime_quarter.readable.datepro}")

Adjusting
Quarter 'full': 2023-01-06 23:09:10.8000


In [ ]:
#mtime = montu.Time('2023-01-31 00:00:00')
mtime = montu.Time('2023-01-22 08:45:00')
next_moon_quarters2(since=mtime,output='datepro')

Closest new': 2023-01-21 20:54:23.1000
Reference: 2023-01-21 20:54:23.1000
	Next quarter 'first': 2023-01-28 15:20:01.7000
	Time to previous: 6.767808315809816
		Stored
Reference: 2023-01-21 20:54:23.1000
	Next quarter 'full': 2023-02-05 18:29:51.7000
	Time to previous: 8.131828685291111
		Stored
Reference: 2023-01-21 20:54:23.1000
	Next quarter 'last': 2023-02-13 16:01:56.3000
	Time to previous: 7.897275378461927
		Stored
Reference: 2023-02-13 16:01:56.3000
	Next quarter 'new': 2023-02-20 07:06:56.9000
	Time to previous: 6.628478765487671
		Stored


{'first': ['2023-01-28 15:20:01.7000', 6.767808315809816, 6.274325734935701],
 'full': ['2023-02-05 18:29:51.7000', 8.131828685291111, 14.406154420226812],
 'last': ['2023-02-13 16:01:56.3000', 7.897275378461927, 22.30342979868874],
 'new': ['2023-02-20 07:06:56.9000', 6.628478765487671, 28.93190856417641]}

In [ ]:
mtime = montu.Time('2023-01-01 08:45:00')
montu.Moon.next_moon_quarters(since=mtime,output='datepro',starting_at='new',numquarters=6)

{'new1': ['2023-01-21 20:54:23.1000', 6.7797199985943735, 20.506517419125885],
 'first1': ['2023-01-28 15:20:01.7000', 6.767808315809816, 27.2743257349357],
 'full1': ['2023-02-05 18:29:51.7000', 8.131828685291111, 35.40615442022681],
 'last1': ['2023-02-13 16:01:56.3000', 7.897275378461927, 43.30342979868874],
 'new2': ['2023-02-20 07:06:56.9000', 6.628478765487671, 49.93190856417641],
 'first2': ['2023-02-27 08:06:50.3000', 7.0415904857218266, 56.97349904989824]}

### Calendar development